In [1]:
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import time
from scipy import optimize
%matplotlib inline

In [2]:
np.random.seed = 420

# Gradient descent method
Since matrix $P$ is given in the problem statement, we will set it uniformly distributed. Let $n = 4$ and $m = 5$.

In [74]:
n = 4
m = 5
P = np.array([np.random.uniform(size=n) for x in np.zeros(m)])
P /= P.sum(axis=0)
c_t = np.array([-np.sum(x * np.log2(x)) for x in P.T])
print(P)

[[ 0.36366239  0.1375436   0.11298746  0.15422615]
 [ 0.06170611  0.20613491  0.270838    0.18208947]
 [ 0.40068035  0.0850128   0.22377749  0.09740263]
 [ 0.11047012  0.31705305  0.30961554  0.06979643]
 [ 0.06348103  0.25425564  0.0827815   0.49648532]]


In [76]:
def f(x):
    #get projection of x
    x_ = euclidean_proj_simplex(x)
    y = P @ x_
    if (np.min(y) <= 0):
        return np.inf
    return c_t @ x_ + np.sum(y * np.log(y) / np.log(2))

def grad_f(x):
    #get projection of x
    x_ = euclidean_proj_simplex(x)
    y = P @ x_
    if (np.min(y) <= 0):
        raise ValueError
    grad = c_t.copy()
    tmp = []
    for i in range(m):
        tmp.append(P[i] * (np.log(P[i] @ x_) + 1) / np.log(2))
    tmp_sum = np.sum(np.array(tmp), axis=0)
    return grad + tmp_sum

In [77]:
def euclidean_proj_simplex(v, s=1):
    n, = v.shape  
    if v.sum() == s and np.alltrue(v >= 0):
        return v
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    rho = np.nonzero(u * np.arange(1, n+1) > (cssv - s))[0][-1]
    theta = (cssv[rho] - s) / (rho + 1.0)
    w = (v - theta).clip(min=0)
    return w

In [78]:
class StoppingCriteria:
    def __init__(self, max_iterations=np.inf, min_grad_norm=0):
        self.max_iterations = max_iterations
        self.min_grad_norm = min_grad_norm
    
    def __call__(self, state):
        cur_iterations = state['iterations']
        cur_grad_norm = np.linalg.norm(state['cur_grad'], ord=2)
        dif_z = np.linalg.norm(state['z'] - state['prev_z'], ord=2)
        return (cur_iterations >= self.max_iterations or cur_grad_norm <= self.min_grad_norm or dif_z <= self.min_grad_norm)

In [90]:
class StepSearchFastestTernary:
    def __init__(self, precision):
        self.precision = precision
        self.left = 0
        self.right = None
        
    def __update_starting_points(self, state, init_kpower=-2):
        k_power = init_kpower
        f = state['f']
        z = state['z']
        grad_f = state['cur_grad']
        dx = - grad_f
        while f(z + 2**k_power * dx) > f(z + 2**(k_power + 1) * dx):
            k_power += 1
        if k_power == init_kpower:
            self.left = 0
        else:
            self.left = 2**(k_power - 1)
        self.right = 2**(k_power + 1)
            
            
    def __call__(self, state):
        f = state['f']
        z = state['z']
        cur_grad = state['cur_grad']
        dx = -cur_grad
        
        self.__update_starting_points(state) # update self.left and self.right
        
        right = self.right
        left = self.left
        
        while True:
            if abs(right - left) < self.precision:
                return (left + right)/2

            left_div = left + (right - left)/3
            right_div = right - (right - left)/3

            f_left = f(z + left_div * dx)
            f_right = f(z + right_div * dx)
            
            if f_left == np.inf:
                right = right_div
            else:
                if f_left < f_right:
                    right = right_div
                else:
                    left = left_div
        

In [85]:
class GradientDescentMethod:
    def __init__(self, t_search, stopping_criteria):
        self.t_search = t_search
        self.stopping_criteria = stopping_criteria
    
    def fit(self, f, grad_f, z_0):
        z = z_0.copy()
        state = dict()
        state['f'] = f
        state['grad_f'] = grad_f
        state['z'] = z
        # hardcoded 
        state['prev_z'] = np.ones(n) / n
        state['iterations'] = 0
        state['time'] = time.time()
        while True:
            state['cur_grad'] = grad_f(state['z'])
            if self.stopping_criteria(state):
                break
            t = self.t_search(state)
            state['prev_z'] = state['z'].copy()
            state['z'] -= t * state['cur_grad']
            #take projection on simplex
            state['z'] = euclidean_proj_simplex(state['z'])
            state['iterations'] += 1
            
        state['time'] = time.time() - state['time']
        return state

In [86]:
stopping_criteria = StoppingCriteria(min_grad_norm=1e-7)
t_search = StepSearchFastestTernary(precision=1e-7)
grad = GradientDescentMethod(t_search=t_search, stopping_criteria=stopping_criteria)
z_0 = np.random.uniform(low=0, high=1, size=n)
z_0 /= np.sum(z_0)
state = grad.fit(f, grad_f, z_0)
print('z_0 = ', z_0)
print('f_min =', f(state['z']))
print('time = ', state['time'])
print('z = ',state['z'])

z_0 =  [ 0.36451531  0.03186695  0.27508598  0.32853177]
f_min = -0.298013392595
time =  0.059381961822509766
z =  [ 0.38330255  0.          0.23675372  0.37994374]


In [87]:
cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1},
        {'type': 'ineq', 'fun': lambda x: x[0]},
        {'type': 'ineq', 'fun': lambda x: x[1]},
       {'type': 'ineq', 'fun': lambda x: x[2]},
       {'type': 'ineq', 'fun': lambda x: x[3]})

In [88]:
print(optimize.minimize(f, z_0, method='SLSQP',
               constraints=cons))

     fun: -0.298012922964378
     jac: array([ -6.03929162e-04,   3.33339423e-02,   5.40405512e-04,
         6.35832548e-05])
 message: 'Optimization terminated successfully.'
    nfev: 18
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([  3.82490710e-01,   4.63496429e-18,   2.37586949e-01,
         3.79922341e-01])
